In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt

from stereoVO.structures import StateBolts, VO_StateMachine
from stereoVO.datasets import KittiDataset
from stereoVO.configs import yaml_parser

from stereoVO.geometry import (DetectionEngine, 
                               filter_matching_inliers, 
                               triangulate_points, 
                               filter_triangulated_points)

In [ ]:
PATH = '../../KITTI/KITTI_gray/dataset/sequences/00/'
CONFIG_PATH =  '../configs/params.yaml'

**Initialise the dataset**

In [ ]:
dataset = KittiDataset(PATH)

dataset.intrinsic = dataset.camera_intrinsic()

**Load the Configs**

In [ ]:
params = yaml_parser(CONFIG_PATH)

**Initialise the State**

In [ ]:
state_num = 0

left_frame, right_frame, ground_truth = dataset[state_num]

#initialise the state
prevState = VO_StateMachine(state_num)

#set frame state
prevState.frames = left_frame, right_frame

**Detction Engine, Matching and Triangulation for first frame**

In [ ]:
detection_engine = DetectionEngine(prevState.frames.left, prevState.frames.right, params)

prevState.matchedPoints, prevState.keypoints, prevState.descriptors = detection_engine.get_matching_keypoints()

prevState.inliers = filter_matching_inliers(prevState.matchedPoints.left,
                                            prevState.matchedPoints.right,
                                            dataset.intrinsic,
                                            params)

prevState.pts3D, reproj_error = triangulate_points(prevState.inliers.left,
                                    prevState.inliers.right,
                                    dataset.PL,
                                    dataset.PR)

args_triangulation = params.geometry.triangulation
prevState.pts3D_Filter, maskTriangulationFilter, ratioFilter = filter_triangulated_points(prevState.pts3D, reproj_error, **args_triangulation)

prevState.InliersFilter = prevState.inliers.left[maskTriangulationFilter], prevState.inliers.right[maskTriangulationFilter]
prevState.ratioTriangulationFilter = ratioFilter

**Initialise the Second State and set it as currState**

In [ ]:
state_num = 1

left_frame, right_frame, ground_truth = dataset[state_num]

#initialise the state
currState = VO_StateMachine(state_num)

#set frame state
currState.frames = left_frame, right_frame

In [ ]:
detection_engine = DetectionEngine(currState.frames.left, currState.frames.right, params)

currState.matchedPoints, currState.keypoints, currState.descriptors = detection_engine.get_matching_keypoints()

currState.inliers = filter_matching_inliers(currState.matchedPoints.left,
                                            currState.matchedPoints.right,
                                            dataset.intrinsic,
                                            params)

currState.pts3D, reproj_error = triangulate_points(currState.inliers.left,
                                    currState.inliers.right,
                                    dataset.PL,
                                    dataset.PR)

args_triangulation = params.geometry.triangulation
currState.pts3D_Filter, maskTriangulationFilter, ratioFilter = filter_triangulated_points(currState.pts3D, reproj_error, **args_triangulation)

currState.InliersFilter = currState.inliers.left[maskTriangulationFilter], currState.inliers.right[maskTriangulationFilter]
currState.ratioTriangulationFilter = ratioFilter